In [85]:
import numpy as np
import math

In [104]:
TRAINING_SAMPLES = 0.75
FEATURES = 1

In [87]:
x_values = np.arange(0, 500, 1e-3)
sinx = np.array([math.sin(i) for i in x_values])
print(sinx.shape)

(500000,)


In [88]:
# split into train and test sets
train_size = int(len(sinx) * TRAINING_SAMPLES)
test_size = len(sinx) - train_size
train, test = sinx[0:train_size], sinx[train_size:len(sinx)]
print(train.shape, test.shape)

((375000,), (125000,))


In [93]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back)]
		dataX.append(a)
		dataY.append(dataset[i + look_back])
	return np.array(dataX), np.array(dataY)

In [98]:
# reshape into X=t and Y=t+1
look_back = 8
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
print (trainX.shape, trainY.shape)

((374991, 8), (374991,))


In [106]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], look_back, FEATURES))
testX = np.reshape(testX, (testX.shape[0], look_back, FEATURES))
print (trainX.shape)

(374991, 8, 1)


In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(look_back, FEATURES)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=["accuracy"])
model.fit(trainX, trainY, validation_data=[testX, testY], epochs=3, batch_size=10, verbose=1)


In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# calculate mean squared error
trainScore = mean_squared_error(trainY[0], trainPredict[:,0])
print('Train Score: %.2f MSE' % (trainScore))
testScore = mean_squared_error(testY[0], testPredict[:,0])
print('Test Score: %.2f MSE' % (testScore))

# In[ ]:


# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.savefig("result.jpg")
plt.show()

